**Install packages:** Install the necessary packages for running the notebook.

In [ ]:
pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /private/var/folders/0c/j4c53d0102lcr2b5rk247yrm0000gn/T/pip-req-build-zrd2r15y
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /private/var/folders/0c/j4c53d0102lcr2b5rk247yrm0000gn/T/pip-req-build-zrd2r15y
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install transformers

In [ ]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install evaluate

In [ ]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install jiwer

In [ ]:
pip install accelerate -U

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.2 MB/s eta 0:00:00


**Setting up PyTorch Device:** Check if CUDA is available and set the device to GPU if possible; otherwise, fall back to CPU.

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


**Dataset Preparation:** Import necessary libraries and define a function to create a dataset from a TSV file, which reads the file, adjusts the file paths, and converts the data into a Hugging Face dataset format.

In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd
from pathlib import Path

base_path = Path('cv-corpus-17.0-2024-03-15/hy-AM')

def create_dataset_from_dataframe(split_name, base_path):
    df = pd.read_csv(base_path / f'{split_name}.tsv', sep='\t')
    clips_path = base_path / 'clips'
    df['path'] = df['path'].apply(lambda x: str(clips_path / x))
    dataset = Dataset.from_pandas(df)
    return dataset

my_common_voice = DatasetDict()

my_common_voice['train'] = create_dataset_from_dataframe('train', base_path)
my_common_voice['test'] = create_dataset_from_dataframe('test', base_path)
my_common_voice['validation'] = create_dataset_from_dataframe('dev', base_path)

**Clean Dataset:** Remove unnecessary columns like 'client_id', 'up_votes', etc., from the dataset

In [ ]:
my_common_voice = my_common_voice.remove_columns(["client_id", "up_votes", "down_votes", "age", "gender", "accents", "variant", "locale", "segment"])

**Print Dataset Overview:** Display the structure of the dataset to confirm the changes and view the remaining columns.

In [ ]:
print(my_common_voice)

DatasetDict({
    train: Dataset({
        features: ['path', 'sentence_id', 'sentence', 'sentence_domain'],
        num_rows: 6180
    })
    test: Dataset({
        features: ['path', 'sentence_id', 'sentence', 'sentence_domain'],
        num_rows: 4281
    })
    validation: Dataset({
        features: ['path', 'sentence_id', 'sentence', 'sentence_domain'],
        num_rows: 4214
    })
})


**Load Feature Extractor and Tokenizer:** Load the Whisper feature extractor and tokenizer from Hugging Face's Transformers library.

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Armenian", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


**Transcription Tokenization and Decoding:** Perform tokenization on a sample text, and decode it to ensure the process retains the original text without any special tokens.

In [ ]:
input_str = my_common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 Հորիզոնին երևում են մերկ ծառեր և տներ։
Decoded w/ special:    <|startoftranscript|><|hy|><|transcribe|><|notimestamps|>Հորիզոնին երևում են մերկ ծառեր և տներ։<|endoftext|>
Decoded w/out special: Հորիզոնին երևում են մերկ ծառեր և տներ։
Are equal:             True


**Print a Sample Entry:** Display a specific example from the training set to verify the data structure and contents.

In [ ]:
print(my_common_voice["train"][0])

{'path': 'cv-corpus-17.0-2024-03-15/hy-AM/clips/common_voice_hy-AM_26466425.mp3', 'sentence_id': '014e1b24fd333a70a23e79c20cd4822a843711386f585a6a61f2bbbd6b796f7a', 'sentence': 'Հորիզոնին երևում են մերկ ծառեր և տներ։', 'sentence_domain': None}


**Define Data Preparation Function:** Create a function to process the audio data: loading, resampling, extracting features using Whisper, and encoding the labels for training.

In [ ]:
import librosa

def load_and_resample_audio(file_path, target_sr=16000):
    audio, _ = librosa.load(file_path, sr=target_sr)
    return {'array': audio, 'sampling_rate': target_sr}

**Apply Data Preparation:** Use the 'map' method to apply the data preparation function across the dataset, ensuring the data is ready for input into the model.

In [ ]:
def prepare_dataset(batch):
    path = batch["path"]
    audio = load_and_resample_audio(path)
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
my_common_voice = my_common_voice.map(prepare_dataset, remove_columns=my_common_voice.column_names["train"])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

**Define Data Collator Class:** This class is designed to handle batching of speech sequences which might have variable lengths. It processes the input features and labels, padding them appropriately for the model training.

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

**Initialize Processor and Data Collator:** Load the Whisper processor and instantiate the custom data collator class using this processor.

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Armenian", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

**Setup Evaluation Metrics:** Import evaluation metrics specifically for word error rate (WER) and character error rate (CER) to be used during model testing.

In [ ]:
import evaluate

metric_wer = evaluate.load("wer")
metric_cer = evaluate.load("cer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric_wer.compute(predictions=pred_str, references=label_str)
    cer = 100 * metric_cer.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}

**Load Pretrained Model for Conditional Generation:** Import the WhisperForConditionalGeneration class from the transformers library and load a pretrained Whisper model configured for generating text in Armenian. The model configuration is also adjusted to set the generation language to Armenian.

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.generation_config.language = "armenian"

**Configure Model Decoding:** Configure the model's decoding behavior by setting forced_decoder_ids to None and clearing any suppress_tokens. These settings adjust how the model generates output, ensuring that it does not force any specific decoder token IDs and does not suppress any tokens during the decoding phase.

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

**Create Logging Directory** Checks if a logging directory exists and if not, it creates one. This directory will be used to store training logs, which are crucial for monitoring the training process and evaluating model performance over time.

In [ ]:
import os

path = "logs/small_25_epoch_48_hours"
if not os.path.exists(path):
    os.makedirs(path)
    logdir = path
else:
    logdir = path

**Define Training Arguments:** Setup various training parameters such as batch sizes, learning rates, and evaluation strategies.

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="whisper-small-hy-48-hours",
    per_device_train_batch_size=16,
    logging_dir=logdir,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=1,
    num_train_epochs=1,
    gradient_checkpointing=True,
    fp16=False,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_strategy = "epoch",
    evaluation_strategy = "epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

**Prepare Model for Training:** Move the model to the designated computing device (GPU or CPU).

In [ ]:
model.to(device)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        

**Initialize Training Setup:** Combine all necessary components like the model, datasets, data collator, and metrics function into the Seq2SeqTrainer for training.

In [ ]:
from transformers import TrainerCallback

class SaveLastModelCallback(TrainerCallback):
    def on_train_end(self, args, state, control, **kwargs):
        trainer.save_model(output_dir=args.output_dir + "/last_epoch")

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=my_common_voice["train"],
    eval_dataset=my_common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[SaveLastModelCallback()],
)

**Training:** Execute the training process using the trainer configuration established in previous cells.

In [ ]:
trainer.train()

/home/user1801/anaconda3/envs/speech/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Epoch,Training Loss,Validation Loss


**Load Pretrained Model:** Load a Whisper model pre-trained for conditional generation with specific configurations set for Armenian language.

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperTokenizer, WhisperFeatureExtractor, WhisperProcessor
import evaluate
import librosa

metric_wer = evaluate.load("wer")
metric_cer = evaluate.load("cer")

model = WhisperForConditionalGeneration.from_pretrained("whisper-small-hy-48-hours/checkpoint-9675")
model.generation_config.language = "armenian"
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Armenian", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("whisper-small-hy-48-hours/checkpoint-9675")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Armenian", task="transcribe")
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        

In [ ]:
def map_to_pred(batch):
    path = batch["path"]
    audio, _ = librosa.load(batch["path"], sr=16000)
    input_features = processor(audio, sampling_rate=16000, return_tensors="pt").input_features
    batch["reference"] = processor.tokenizer._normalize(batch['sentence'])

    with torch.no_grad():
        predicted_ids = model.generate(input_features.to("cuda"))[0]
    transcription = processor.decode(predicted_ids)
    batch["prediction"] = processor.tokenizer._normalize(transcription)
    return batch

result = my_common_voice["test"].map(map_to_pred)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
print("Test WER: {:.3f}".format(100 * metric_wer.compute(references=result["reference"], predictions=result["prediction"])))
print("Test CER: {:.3f}".format(100 * metric_cer.compute(references=result["reference"], predictions=result["prediction"])))

Test WER: 46.053
Test CER: 10.702


In [ ]:
model.eval()

def transcribe_audio(audio_file):
    audio, sampling_rate = librosa.load(audio_file, sr=16000, mono=True)
    inputs = feature_extractor(audio, return_tensors="pt", sampling_rate=sampling_rate)
    inputs = inputs.to(device)

    with torch.no_grad():
        predictions = model.generate(inputs.input_features)

    text = tokenizer.batch_decode(predictions, skip_special_tokens=True)[0]
    return text

In [ ]:
result = transcribe_audio('cv-corpus-16.1-2023-12-06/hy-AM/clips/common_voice_hy-AM_39509489.mp3')
print(result)

Պատմողն աշխատանք է ստանում գրադարանում։


In [ ]:
result = transcribe_audio('cv-corpus-16.1-2023-12-06/hy-AM/clips/common_voice_hy-AM_39517769.mp3')
print(result)

Այս պատճառով, որոշ ծրագրային բագեր չեն հայտնաբերվում մինչև ծրագիրը չի տեստավորվում։


In [ ]:
result = transcribe_audio('cv-corpus-16.1-2023-12-06/hy-AM/clips/common_voice_hy-AM_39427421.mp3')
print(result)

Նա կոչ չէր անում սպիտակամորդների դեմ պայքարել՝ մարգարեները չէին խոսումները անունից։


In [ ]:
result = transcribe_audio('cv-corpus-16.1-2023-12-06/hy-AM/clips/common_voice_hy-AM_39295677.mp3')
print(result)

Թեկլեն սկսեց զգալ ցամակ աչքերով, բայց այնպիսի աղեկտուլ խոսքերով, ուրնույնի սկոր։


In [ ]:
import yt_dlp
import os


def download_youtube_audio(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': '%(id)s.%(ext)s',
        'quiet': False
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=True)
        filename = ydl.prepare_filename(info_dict)
        base, ext = os.path.splitext(filename)
        return base + '.mp3'

audio_file = download_youtube_audio("https://www.youtube.com/watch?v=-OhScYMGqvc")

def segment_audio(audio_file, segment_length=30, overlap=5, sample_rate=16000):
    audio, sr = librosa.load(audio_file, sr=sample_rate, mono=True)
    total_length = librosa.get_duration(y=audio, sr=sr)

    start = 0
    while start + segment_length <= total_length:
        end = start + segment_length
        yield audio[int(start * sr):int(end * sr)]
        start += (segment_length - overlap)

def transcribe_segments(audio_file):
    texts = []
    previous_text = ""
    for segment in segment_audio(audio_file, overlap=5):
        inputs = feature_extractor(segment, return_tensors="pt", sampling_rate=16000)
        inputs = inputs.to(device)

        with torch.no_grad():
            predictions = model.generate(inputs.input_features)

        current_text = tokenizer.batch_decode(predictions, skip_special_tokens=True)[0]

        if previous_text:
            overlap_index = current_text.find(previous_text.split()[-1])
            if overlap_index != -1:
                current_text = current_text[overlap_index + len(previous_text.split()[-1]):].strip()

        texts.append(current_text)
        previous_text = current_text

    return " ".join(texts)

result = transcribe_segments(audio_file)
print(result)

[youtube] Extracting URL: https://www.youtube.com/watch?v=-OhScYMGqvc
[youtube] -OhScYMGqvc: Downloading webpage
[youtube] -OhScYMGqvc: Downloading ios player API JSON
[youtube] -OhScYMGqvc: Downloading android player API JSON


[youtube] -OhScYMGqvc: Downloading m3u8 information
[info] -OhScYMGqvc: Downloading 1 format(s): 251
[download] Destination: -OhScYMGqvc.webm
[download] 100% of    3.10MiB in 00:00:00 at 7.28MiB/s     
[ExtractAudio] Destination: -OhScYMGqvc.mp3
Deleting original file -OhScYMGqvc.webm (pass -k to keep)
Արհեստական բանականթյուն, խելացի ռոբոտներ, չաց ջիփիթիտ, զգացել եք վերջերց միջքան ինքոզում արհեստական բանականթյուն մասին։ողչույն, ես ունարենը զգացելենք և որոշել ենք ստեղծել հաղորդաշար, որի ընթացքում կոգոսեանք արհեստական բանականթյունը շատերը պատկերաստում են այսպես… Պեց նավ, ամեն դեպքում ինչը արհեստական բանականությունը, մի լովը… արհեստական բանականություն, գիտություն, որով մեքենաները դառնում են խելացի…Նեսած կլինեակ, որ արհեստական բանականությունը վերասնալուէ է բալորմասնագիտությունները, կամոր մարմարկանց կայլիկ այլևս չինինելու՞՞թէէէէէէէէէէէէէէէէ� Դեը արհեստական բանականությունն այսօր ամենուր է։բժշկության մեջ բարձ վիրահատություններ անով ռոբոտներ են ստեացվում, արհեստական բանականությամբ ստեացված նկա